### Agent Ideation with AutoGen

Let's build an agent which uses two LLMs to debate ideas based on an instruction from the user. For this agent we'll use Microsoft's AutoGen agentic framework and OpenAI.

Let's suppress as much of the warning output as we can.

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

We'll install the two AutoGen libraries that we need.

In [ ]:
%pip -q install -U autogen-agentchat
%pip -q install autogen-ext[openai]    

We can now import the libraries we'll be using in our agent

In [ ]:
import asyncio, os

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


We'll fetch the OpenAI environment variable and set up a handle to OpenAI.

In [ ]:
api_key = os.environ['OPENAI_API_KEY']
# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=api_key,
)

We'll create two agents, one to generate ideas and teh second to review them.

In [ ]:
primary_agent = AssistantAgent(
    "Alice",
    model_client=model_client,
    system_message="You are a participant in an ideation and feedback session. You will be provided with a problem statement and asked to generate ideas. Your ideas will be\
    reviwed by another participant and then you together will narrow down ideas by debating over them. Respond with 'FINALIZE' when you have a final idea.",
)

critic_agent = AssistantAgent(
    "Bob",
    model_client=model_client,
    system_message="You are a participant in an ideation and feedback session. Your teammate will be provide some ideas that you need to review with your \
        teammate and narrow down ideas by debating over them. Respond with 'FINALIZE' when you have a final idea.",
)

We can now create the chain of agents and specify the termination condition

In [ ]:
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=TextMentionTermination("FINALIZE"))

We've got our chain of agents (the 'team') set up so let's get them started ideating.  
We'll display their conversation as they go.

In [ ]:
result = await team.run(task="Generate ideas for applications of AI in a veterenarian practice.")
for message in result.messages:
        print("--------------------")
        print(message.source)
        print("--------------------")
        print(message.content)
